Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.

In [74]:
!apt install mongodb > log

In [26]:
!service mongodb start

 * Starting database mongodb
   ...done.


In [27]:
from pymongo import MongoClient

In [28]:
client = MongoClient('127.0.0.1', 27017)

In [29]:
print(client.list_database_names()) 

['admin', 'config', 'local']


In [30]:
from google.colab import files  #импортируем данные из дз№3
uploaded = files.upload()

Saving Vacancy_HH.csv to Vacancy_HH (1).csv


In [31]:
import pandas as pd
df_new_hh = pd.read_csv('Vacancy_HH.csv',sep=',') #открыли файл


In [12]:
#df_new_hh

In [32]:
#Создаём ссылку на базу данных 
my_db = client['db_vacancies_matematik']
collection_vacancies = my_db['vacancies']

In [33]:
len(df_new_hh)

100

In [44]:
columns_df = df_new_hh.columns

for index, row in df_new_hh.iterrows():
    vacancy_dict ={}
    for column in columns_df:
        vacancy_dict[column] = row[column]
    #print(vacancy_dict)
    my_db.vacancies.insert_one(vacancy_dict)

In [45]:
client.list_database_names()



['admin', 'config', 'db_vacancies_matematik', 'local']

In [46]:
num_vacancies = my_db.vacancies.count_documents({})
print(num_vacancies)

100


In [47]:
from bs4 import BeautifulSoup as bs
import requests
from time import sleep
from pprint import pprint

Добавляем новые вакансии в Базу Данных


In [48]:
def add_vacancies(url, headers):
  response = requests.get(url = url, headers = headers)
  print(response.status_code)
  dom = bs(response.text, 'html.parser')
  tag_vacancy = dom.find_all('div', class_= 'vacancy-serp-item-body__main-info')
  print(f'считано - {len(tag_vacancy)} вакансий')
  site_name = 'hh.ru'
  number_new_vacancies = 0
  for vacancy in tag_vacancy:
    vacancy_link =  vacancy.find('a', attrs={'class': 'serp-item__title', 'data-qa': 'serp-item__title'}).get('href')
    if my_db.vacancies.count_documents({'ссылка на вакансию': vacancy_link}) == 0:
      vacancy_title = vacancy.find('a', attrs={'class': 'serp-item__title', 'data-qa': 'serp-item__title'}).string
      if vacancy.find('span', attrs={'class': 'bloko-header-section-3', 'data-qa': 'vacancy-serp__vacancy-compensation'}) == None:
        min_compensation, max_compensation, currency = None, None, None
      else:
        compensation = vacancy.find('span', attrs={'class': 'bloko-header-section-3', 'data-qa': 'vacancy-serp__vacancy-compensation'}).text.replace(' ', '').split()
        if compensation[0] == 'от':
          min_compensation = int(compensation[1])
          max_compensation = None
          currency = compensation[2]
        elif compensation[0] == 'до':
          min_compensation = None
          max_compensation = int(compensation[1])
          currency = compensation[2]
        else:
          min_compensation = int(compensation[0])
          max_compensation = int(compensation[2])
          currency = compensation[3]
      my_db.vacancies.insert_one({
          'должность': vacancy_title,
          'минимальная зарплата' : min_compensation,
          'максимальная зарплата' : max_compensation,
          'валюта' : currency,
          'ссылка на вакансию' : vacancy_link,
          'сайт' : site_name
           })
      number_new_vacancies +=1
  print(f'добавлено {number_new_vacancies} новых вакансий')  
      

In [51]:
add_vacancies('https://hh.ru/vacancies/matematik', {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36',
})

200
считано - 56 вакансий
добавлено 41 новых вакансий


In [52]:
num_vacancies = my_db.vacancies.count_documents({})
print(num_vacancies)

141


Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты).

In [75]:
def job_search_by_salary(salary_min, salary_max):
    count = 0 
    for i in my_db.vacancies.find({'зарплата от': {'$gt': salary_min}, 'зарплата до': {'$gt': salary_max}}):
        pprint(i)
        print('')
        count += 1
    return f'найдено {count} подходящих вакансий'

In [77]:
job_search_by_salary(60000, 100000)

'найдено 0 подходящих вакансий'

In [58]:
for doc in my_db.vacancies.find({'$and': [{'минимальная зарплата':{'$gt': 60000}}, {'максимальная зарплата':{'$gt': 100000}}]}):
  pprint(doc)

{'_id': ObjectId('63274384ca1755779e049778'),
 'валюта': 'руб.',
 'должность': 'Математик - программист C/Python',
 'максимальная зарплата': 140000.0,
 'минимальная зарплата': 90000.0,
 'сайт': 'hh.ru',
 'ссылка на вакансию': 'https://hh.ru/vacancy/69036019?query=%D0%BC%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D0%BA'}
{'_id': ObjectId('63274384ca1755779e049785'),
 'валюта': 'руб.',
 'должность': 'Преподаватель математики',
 'максимальная зарплата': 120000.0,
 'минимальная зарплата': 80000.0,
 'сайт': 'hh.ru',
 'ссылка на вакансию': 'https://hh.ru/vacancy/69494784?query=%D0%BC%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D0%BA'}
{'_id': ObjectId('63274384ca1755779e049788'),
 'валюта': 'руб.',
 'должность': 'Аналитик',
 'максимальная зарплата': 160000.0,
 'минимальная зарплата': 140000.0,
 'сайт': 'hh.ru',
 'ссылка на вакансию': 'https://hh.ru/vacancy/69589543?query=%D0%BC%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D0%BA'}
{'_id': ObjectId('63274384ca1755779e04978e'),
 'валюта': 'руб.',
 'д

In [82]:
def chek_salary(max_salary):
    count=0
    salary=my_db.vacancies.find({'min_salary':{'$gt':max_salary},'max_salary':{'$gt':max_salary}})
    for i in salary:
        count+=1
        print(i['name'], 'может получать зп больше', max_salary)
    return f'найдено {count} подходящих вакансий'
    

In [83]:
chek_salary(60000)

'найдено 0 подходящих вакансий'

что-то функции не работают((

Любая аналитика. Например matching ваканский с разных площадок

In [67]:
for doc in my_db.vacancies.find({ 'должность': { "$regex": 'учитель'} }):
  pprint(doc)

{'_id': ObjectId('63274384ca1755779e049779'),
 'валюта': 'руб.',
 'должность': 'Частный учитель по математике и дз к ученику 6 класса',
 'максимальная зарплата': nan,
 'минимальная зарплата': 80000.0,
 'сайт': 'hh.ru',
 'ссылка на вакансию': 'https://hh.ru/vacancy/69232202?query=%D0%BC%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D0%BA'}
{'_id': ObjectId('63274384ca1755779e0497ab'),
 'валюта': 'руб.',
 'должность': 'Частный учитель по математике и дз к ученику 6 класса',
 'максимальная зарплата': nan,
 'минимальная зарплата': 80000.0,
 'сайт': 'hh.ru',
 'ссылка на вакансию': 'https://hh.ru/vacancy/69232202?query=%D0%BC%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D0%BA'}


In [85]:
for doc in my_db.vacancies.find({ 'должность': { "$regex":'преподаватель' } }):
  pprint(doc)

{'_id': ObjectId('63274599ca1755779e0497f0'),
 'валюта': 'руб.',
 'должность': 'Старший преподаватель математики',
 'максимальная зарплата': None,
 'минимальная зарплата': 55000,
 'сайт': 'hh.ru',
 'ссылка на вакансию': 'https://hh.ru/vacancy/68642982?query=%D0%BC%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D0%BA'}
